In [1]:
import numpy as np
import pandas as pd

In [9]:
scores = pd.read_csv('summary_scores.csv')

In [10]:
scores.head()

,id,roa,recall,precision,actual_id,pred_id,cov,mdr,subject,mvc,attempt
0,0,1.000000,1.000000,1.000000,88,0,0.139112,8.642673e-07,1,10,1
1,1,0.995885,1.000000,0.995885,61,1,0.145394,9.052455e-07,1,10,1
2,2,0.543974,1.000000,0.543974,223,231,0.396034,1.143664e-06,1,10,1
3,3,0.448052,0.992806,0.449511,252,231,0.396034,1.143664e-06,1,10,1
4,4,1.000000,1.000000,1.000000,3,3,0.143645,9.387732e-07,1,10,1


In [11]:
scores.drop(columns=['id', 'actual_id', 'pred_id'], inplace=True)

In [12]:
scores.head()

,roa,recall,precision,cov,mdr,subject,mvc,attempt
0,1.000000,1.000000,1.000000,0.139112,8.642673e-07,1,10,1
1,0.995885,1.000000,0.995885,0.145394,9.052455e-07,1,10,1
2,0.543974,1.000000,0.543974,0.396034,1.143664e-06,1,10,1
3,0.448052,0.992806,0.449511,0.396034,1.143664e-06,1,10,1
4,1.000000,1.000000,1.000000,0.143645,9.387732e-07,1,10,1


In [13]:
# Fix MDR
# Was: len / samples / fs
# Should be: len / (samples / fs)
# Multiply by samples and fs then divide by (samples / fs)
fs = 4096
seconds = 16
n_samples = fs * seconds
scores['mdr'] = (scores['mdr'] * n_samples * fs) / (n_samples / fs)

In [14]:
scores.head()

,roa,recall,precision,cov,mdr,subject,mvc,attempt
0,1.000000,1.000000,1.000000,0.139112,14.5000,1,10,1
1,0.995885,1.000000,0.995885,0.145394,15.1875,1,10,1
2,0.543974,1.000000,0.543974,0.396034,19.1875,1,10,1
3,0.448052,0.992806,0.449511,0.396034,19.1875,1,10,1
4,1.000000,1.000000,1.000000,0.143645,15.7500,1,10,1


In [34]:
# Discard results outside of MDR of [6; 40] and above CoV of 0.5
scores = scores[(scores['mdr'] < 40) & (scores['mdr'] > 6) & (scores['cov'] < 0.5)]
len(scores)

2377

In [36]:
# Make a copy
mvc_raw_scores = scores.copy()
# Drop unnecessary columns
mvc_raw_scores.drop(columns=['subject', 'attempt'], inplace=True)
mvc_grouped = mvc_raw_scores.groupby('mvc').agg(['mean', 'std']).round(2)
mvc_grouped.head()

roa       recall       precision         cov          mdr      
     mean   std   mean   std      mean   std  mean   std   mean   std
mvc                                                                  
10   0.83  0.22   0.99  0.03      0.84  0.22  0.23  0.11  15.41  3.70
30   0.95  0.13   1.00  0.02      0.95  0.13  0.18  0.07  21.44  8.03
50   0.97  0.10   1.00  0.03      0.97  0.10  0.16  0.05  23.21  8.09

In [46]:
count_raw_scores = scores.copy()
count_grouped = count_raw_scores.groupby(['subject', 'attempt', 'mvc']).size()
count_grouped.head(10)

subject  attempt  mvc
1        1        10     44
         2        10     46
         3        10     47
         4        10     47
         5        10     47
2        1        30     29
         2        30     29
         3        30     27
         4        30     27
         5        30     29
dtype: int64

In [48]:
count_grouped_mvc = count_grouped.groupby('mvc').agg(['mean', 'std']).round(2)
count_grouped_mvc.head()

,mean,std
mvc,,
10,42.12,2.74
30,29.48,5.27
50,23.48,2.63


In [52]:
times = pd.read_csv('summary_times.csv')
times.drop(columns=['id', 'subject', 'attempt'], inplace=True)
times.head()

,time,mvc
0,781.525734,10
1,782.151844,10
2,773.107493,10
3,783.372622,10
4,936.699068,10


In [54]:
times_grouped = times.groupby('mvc').agg(['mean', 'std']).round(2)
times_grouped

time       
        mean    std
mvc                
10    809.27  69.00
30    813.22  17.13
50   1092.42  17.68

In [55]:
mvc_grouped.to_latex()

'\\begin{tabular}{lrrrrrrrrrr}\n\\toprule\n{} & \\multicolumn{2}{l}{roa} & \\multicolumn{2}{l}{recall} & \\multicolumn{2}{l}{precision} & \\multicolumn{2}{l}{cov} & \\multicolumn{2}{l}{mdr} \\\\\n{} &  mean &   std &   mean &   std &      mean &   std &  mean &   std &   mean &   std \\\\\nmvc &       &       &        &       &           &       &       &       &        &       \\\\\n\\midrule\n10  &  0.83 &  0.22 &   0.99 &  0.03 &      0.84 &  0.22 &  0.23 &  0.11 &  15.41 &  3.70 \\\\\n30  &  0.95 &  0.13 &   1.00 &  0.02 &      0.95 &  0.13 &  0.18 &  0.07 &  21.44 &  8.03 \\\\\n50  &  0.97 &  0.10 &   1.00 &  0.03 &      0.97 &  0.10 &  0.16 &  0.05 &  23.21 &  8.09 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [57]:
count_grouped_mvc.to_latex()

'\\begin{tabular}{lrr}\n\\toprule\n{} &   mean &   std \\\\\nmvc &        &       \\\\\n\\midrule\n10  &  42.12 &  2.74 \\\\\n30  &  29.48 &  5.27 \\\\\n50  &  23.48 &  2.63 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [58]:
times_grouped.to_latex()

'\\begin{tabular}{lrr}\n\\toprule\n{} & \\multicolumn{2}{l}{time} \\\\\n{} &     mean &    std \\\\\nmvc &          &        \\\\\n\\midrule\n10  &   809.27 &  69.00 \\\\\n30  &   813.22 &  17.13 \\\\\n50  &  1092.42 &  17.68 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [62]:
merged = mvc_grouped.merge(count_grouped_mvc, on='mvc').merge(times_grouped, on='mvc')
merged.head()

,"(roa, mean)","(roa, std)","(recall, mean)","(recall, std)","(precision, mean)","(precision, std)","(cov, mean)","(cov, std)","(mdr, mean)","(mdr, std)",mean,std,"(time, mean)","(time, std)"
mvc,,,,,,,,,,,,,,
10,0.83,0.22,0.99,0.03,0.84,0.22,0.23,0.11,15.41,3.70,42.12,2.74,809.27,69.00
30,0.95,0.13,1.00,0.02,0.95,0.13,0.18,0.07,21.44,8.03,29.48,5.27,813.22,17.13
50,0.97,0.10,1.00,0.03,0.97,0.10,0.16,0.05,23.21,8.09,23.48,2.63,1092.42,17.68


In [63]:
merged.columns = ['roa_mean', 'roa_std', 'sens_mean', 'sens_std', 'prec_mean', 'prec_std', 'cov_mean', 'cov_std', 'mdr_mean', 'mdr_std', 'mu_mean', 'mu_std', 'time_mean', 'time_std']
merged

,roa_mean,roa_std,sens_mean,sens_std,prec_mean,prec_std,cov_mean,cov_std,mdr_mean,mdr_std,mu_mean,mu_std,time_mean,time_std
mvc,,,,,,,,,,,,,,
10,0.83,0.22,0.99,0.03,0.84,0.22,0.23,0.11,15.41,3.70,42.12,2.74,809.27,69.00
30,0.95,0.13,1.00,0.02,0.95,0.13,0.18,0.07,21.44,8.03,29.48,5.27,813.22,17.13
50,0.97,0.10,1.00,0.03,0.97,0.10,0.16,0.05,23.21,8.09,23.48,2.63,1092.42,17.68


In [74]:
merged.to_csv('processed_results.csv')

In [116]:
tab = pd.read_csv('processed_results.csv')
tab.head()

,mvc,roa_mean,roa_std,sens_mean,sens_std,prec_mean,prec_std,cov_mean,cov_std,mdr_mean,mdr_std,mu_mean,mu_std,time_mean,time_std
0,10,0.83,0.22,0.99,0.03,0.84,0.22,0.23,0.11,15.41,3.70,42.12,2.74,809.27,69.00
1,30,0.95,0.13,1.00,0.02,0.95,0.13,0.18,0.07,21.44,8.03,29.48,5.27,813.22,17.13
2,50,0.97,0.10,1.00,0.03,0.97,0.10,0.16,0.05,23.21,8.09,23.48,2.63,1092.42,17.68


In [117]:
tab['MVC (\%)'] = tab['mvc']
tab['RoA'] = '$' + tab['roa_mean'].astype(str) + '\pm' + tab['roa_std'].astype(str) + '$'
tab['Sensitivity'] = '$' + tab['sens_mean'].astype(str) + '\pm' + tab['sens_std'].astype(str) + '$'
tab['Precision'] = '$' + tab['prec_mean'].astype(str) + '\pm' + tab['prec_std'].astype(str) + '$'
tab['CoV'] = '$' + tab['cov_mean'].astype(str) + '\pm' + tab['cov_std'].astype(str) + '$'
tab['MDR'] = '$' + tab['mdr_mean'].astype(str) + '\pm' + tab['mdr_std'].astype(str) + '$'
tab['Identified MUs'] = '$' + tab['mu_mean'].astype(str) + '\pm' + tab['mu_std'].astype(str) + '$'
tab['Running Time [s]'] = '$' + tab['time_mean'].astype(str) + '\pm' + tab['time_std'].astype(str) + '$'
tab.drop(columns=['roa_mean', 'roa_std', 'sens_mean', 'sens_std', 'prec_mean', 'prec_std', 'cov_mean', 'cov_std', 'mdr_mean', 'mdr_std', 'mu_mean', 'mu_std', 'time_mean', 'time_std', 'mvc'], inplace=True)

In [118]:
tab

,MVC (\%),RoA,Sensitivity,Precision,CoV,MDR,Identified MUs,Running Time [s]
0,10,$0.83\pm0.22$,$0.99\pm0.03$,$0.84\pm0.22$,$0.23\pm0.11$,$15.41\pm3.7$,$42.12\pm2.74$,$809.27\pm69.0$
1,30,$0.95\pm0.13$,$1.0\pm0.02$,$0.95\pm0.13$,$0.18\pm0.07$,$21.44\pm8.03$,$29.48\pm5.27$,$813.22\pm17.13$
2,50,$0.97\pm0.1$,$1.0\pm0.03$,$0.97\pm0.1$,$0.16\pm0.05$,$23.21\pm8.09$,$23.48\pm2.63$,$1092.42\pm17.68$


In [121]:
print(tab[['MVC (\%)', 'Identified MUs', 'RoA', 'Sensitivity', 'Precision', 'CoV', 'MDR']].to_latex(index=False, escape=False))

\begin{tabular}{rllllll}
\toprule
 MVC (\%) &  Identified MUs &            RoA &    Sensitivity &      Precision &            CoV &             MDR \\
\midrule
       10 &  $42.12\pm2.74$ &  $0.83\pm0.22$ &  $0.99\pm0.03$ &  $0.84\pm0.22$ &  $0.23\pm0.11$ &   $15.41\pm3.7$ \\
       30 &  $29.48\pm5.27$ &  $0.95\pm0.13$ &   $1.0\pm0.02$ &  $0.95\pm0.13$ &  $0.18\pm0.07$ &  $21.44\pm8.03$ \\
       50 &  $23.48\pm2.63$ &   $0.97\pm0.1$ &   $1.0\pm0.03$ &   $0.97\pm0.1$ &  $0.16\pm0.05$ &  $23.21\pm8.09$ \\
\bottomrule
\end{tabular}



In [122]:
print(tab[['MVC (\%)', 'Running Time [s]']].to_latex(index=False, escape=False))

\begin{tabular}{rl}
\toprule
 MVC (\%) &   Running Time [s] \\
\midrule
       10 &    $809.27\pm69.0$ \\
       30 &   $813.22\pm17.13$ \\
       50 &  $1092.42\pm17.68$ \\
\bottomrule
\end{tabular}

